In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sheet Music Analysis with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fdocument-processing%2Fsheet_music.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://goo.gle/4jjsLV9">
      <img width="32px" src="https://cdn.qwiklabs.com/assets/gcp_cloud-e3a77215f0b8bfa9b3f611c0d2208c7e8708ed31.svg" alt="Google Cloud logo"><br> Open in  Cloud Skills Boost
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            


| Author |
| --- |
| [Holt Skinner](https://github.com/holtskinner) |

## Overview

[Sheet Music](https://en.wikipedia.org/wiki/Sheet_music) is the primary form of music notation used by composers and performers across the world. These pages contain information about the lyrics, pitches, rhythms, composer, text author, composition date, among others.

This notebook illustrates using Gemini to extract this metadata from sheet music PDFs.

These prompts and documents were demonstrated in the [Google Cloud Next 2024 session "What's next with Gemini: Driving business impact with multimodal use cases"](https://www.youtube.com/watch?v=DqH1R9Pk5RI).


## Getting Started


### Install Google Gen AI SDK for Python

In [2]:
%pip install --upgrade -q google-genai PyPDF2

Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [3]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
# Use the environment variable if the user doesn't provide Project ID.
import os

from google import genai

PROJECT_ID = "qwiklabs-gcp-04-4be09a7f8e06"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "europe-west4")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries


In [5]:
import json

from IPython.display import Markdown, display
import PyPDF2
from google.genai.types import (
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    Tool,
)

### Load the Gemini model

Gemini is a multimodal model that supports multimodal prompts. You can include text, image(s), PDFs, audio, and video in your prompt requests.

In [6]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

config = GenerateContentConfig(
    temperature=1.0,
    max_output_tokens=8192,
    safety_settings=[
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
        )
    ],
    system_instruction="You are an expert in musicology and music history.",
    tools=[Tool(google_search=GoogleSearch())],
)

## Extract Structured Metadata from Sheet Music PDF

For this example, we will be using the popular classical music book [24 Italian Songs and Arias of the 17th and 18th Centuries](https://imslp.org/wiki/24_Italian_Songs_and_Arias_of_the_17th_and_18th_Centuries_(Various)), and extracting metadata about each song in the book.

In [7]:
sheet_music_pdf_uri = "gs://github-repo/use-cases/sheet-music/24ItalianSongs.pdf"

sheet_music_extraction_prompt = """The following document is a book of sheet music. Your task is to output structured metadata about every piece of music in the document.

Correct any mistakes that are in the document and fill in missing information when not present in the document.

Include the following details:

Title
Composer with lifetime
Tempo Marking
A description of the piece
"""

# Send to Gemini
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        sheet_music_extraction_prompt,
        # Load file directly from Google Cloud Storage
        Part.from_uri(
            file_uri=sheet_music_pdf_uri,
            mime_type="application/pdf",
        ),
    ],
    config=config,
)

# Display results
display(Markdown(response.text))

Here is the metadata for each piece of music, extracted from the sheet music document:

1.  **Title:** Per la gloria d'adorarvi
    *   **Composer:** Giovanni Battista Bononcini (1672-1750)
    *   **Tempo Marking:** Andante
    *   **Description:** A composition from the opera "Griselda," where the lyrics express a heart's desire to adore and cherish, even in pain and unrequited love.

2.  **Title:** Amarilli, mia bella
    *   **Composer:** Giulio Caccini (1546-1618)
    *   **Tempo Marking:** Moderato affettuoso
    *   **Description:** A madrigal expressing the doubts and uncertainties of love.

3.  **Title:** Alma del core
    *   **Composer:** Antonio Caldara (1670-1736)
    *   **Tempo Marking:** Tempo di Minuetto
    *   **Description:** A song about the spirit of beauty and a constant, faithful love.

4.  **Title:** Come raggio di sol
    *   **Composer:** Antonio Caldara (1670-1736)
    *   **Tempo Marking:** Sostenuto
    *   **Description:** An aria that describes a beam of sunlight over a swelling wave, transitioning to a somber reflection on fate and hidden storms in the ocean's depths.

5.  **Title:** Sebben, crudele
    *   **Composer:** Antonio Caldara (1670-1736)
    *   **Tempo Marking:** Allegretto grazioso
    *   **Description:** A canzonetta expressing unwavering love despite cruel treatment.

6.  **Title:** Vittoria, mio core!
    *   **Composer:** Giacomo Carissimi (1605-1674)
    *   **Tempo Marking:** Allegro con brio
    *   **Description:** A cantata, is a declaration of victory and freedom from love's torments, transitioning into a reflection on past sorrow and deception.

7.  **Title:** Danza, danza, fanciulla gentile
    *   **Composer:** Francesco Durante (1684-1755)
    *   **Tempo Marking:** Allegro con spirito
    *   **Description:** An arietta, is a call to dance and enjoy life's fleeting moments.

8.  **Title:** Vergin, tutto amor
    *   **Composer:** Francesco Durante (1684-1755)
    *   **Tempo Marking:** Largo religioso
    *   **Description:** A preghiera (prayer) is a heartfelt plea to the Virgin Mary, seeking solace and compassion.

9.  **Title:** Caro mio ben
    *   **Composer:** Giuseppe Giordani (Giordanello) (1744-1798)
    *   **Tempo Marking:** Larghetto
    *   **Description:** An arietta, is a declaration of love.

10. **Title:** O del mio dolce ardor
    *   **Composer:** Christoph Willibald von Gluck (1714-1787)
    *   **Tempo Marking:** Moderato
    *   **Description:** An aria that speaks of a heart longing for a beloved.

11. **Title:** Che fiero costume
    *   **Composer:** Giovanni Legrenzi (1626-1690)
    *   **Tempo Marking:** Allegretto con moto
    *   **Description:** An arietta is a reflection on the torment caused by love.

12. **Title:** Pur dicesti, o bocca bella
    *   **Composer:** Antonio Lotti (1667-1740)
    *   **Tempo Marking:** Allegretto grazioso
    *   **Description:** An arietta, is an exploration of beauty and alluring sweetness.

13. **Title:** Il mio bel foco
    *   **Composer:** Benedetto Marcello (1686-1739)
    *   **Tempo Marking:** Recitativo
    *   **Description:** A recitative and aria that expresses an unwavering love and devotion.

14. **Title:** Lasciatemi morire!
    *   **Composer:** Claudio Monteverdi (1567-1643)
    *   **Tempo Marking:** Lento
    *   **Description:** A canto from the opera "Ariana," is a lament and plea to be left to die.

15. **Title:** Nel cor più non mi sento
    *   **Composer:** Giovanni Paisiello (1740-1816)
    *   **Tempo Marking:** Andantino
    *   **Description:** An arietta that speaks to a sense of dormancy.

16. **Title:** Se tu m'ami, se sospiri
    *   **Composer:** Giovanni Battista Pergolesi (1710-1736)
    *   **Tempo Marking:** Andantino
    *   **Description:** An arietta that speaks of love and sorrow.

17. **Title:** Nina
    *   **Composer:** Attributed to Giovanni Battista Pergolesi (1710-1736), actually by Legrenzio Vincenzo Ciampi (1719-?)
    *   **Tempo Marking:** Andantino
    *   **Description:** A canzonetta about waking Nina.

18. **Title:** Già il sole dal Gange
    *   **Composer:** Alessandro Scarlatti (1659-1725)
    *   **Tempo Marking:** Allegro giusto
    *   **Description:** A canzonetta about the launch of the sun.

19.  **Title:** Le Violette
    *   **Composer:** Alessandro Scarlatti (1659-1725)
    *   **Tempo Marking:** Allegretto
    *   **Description:** A canzone about violets.

20. **Title:** O cessate di piagarmi
    *   **Composer:** Alessandro Scarlatti (1659-1725)
    *   **Tempo Marking:** Andante con moto
    *   **Description:** An arietta.

21. **Title:** Se Florindo è fedele
    *   **Composer:** Alessandro Scarlatti (1659-1725)
    *   **Tempo Marking:** Allegretto grazioso, moderato assai
    *   **Description:** An arietta.

22. **Title:** Pietà, Signore!
    *   **Composer:** Alessandro Stradella (1645?-1682?)
    *   **Tempo Marking:** Andantino
    *   **Description:** A sacred song pleading for mercy.

23. **Title:** Tu lo sai
    *   **Composer:** Giuseppe Torelli (1650-1703)
    *   **Tempo Marking:** Andantino
    *   **Description:** About asking they heart.


You can see that Gemini extracted all of the relevant fields from the document.

### Song Identification with Audio

Now, let's try something more challenging, identifying a song being performed based on the sheet music. We have an audio clip of Holt Skinner performing one of the songs in the book, and we will ask Gemini to identify it based on the sheet music PDF.

In [8]:
song_identification_prompt = """Based on the sheet music PDF, what song is in the audio clip? Explain how you made the decision."""

# Load PDF file
pdf_part = Part.from_uri(
    file_uri=sheet_music_pdf_uri,
    mime_type="application/pdf",
)

audio_part = Part.from_uri(
    file_uri="gs://github-repo/use-cases/sheet-music/24ItalianClip.mp3",
    mime_type="audio/mpeg",
)

# Send to Gemini
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[pdf_part, audio_part, song_identification_prompt],
    config=config,
)

# Display results
display(Markdown(response.text))

The song in the audio clip is "Sebben, crudele" by Antonio Caldara.

I made this decision by listening to the audio clip and recognizing the melody as being from the song "Sebben, crudele". This song is number 6 in the sheet music PDF that you provided. The lyrics "Sebben, crudele, mi fai languir, sempre fedele, sempre fedele ti voglio amar" align with the audio clip provided.


### Edit PDF Metadata

Next, we'll use the output from Gemini to edit the metadata of a PDF containing one song, which can make it easier to organize this file in sheet music applications.

We'll adjust the prompt slightly and set the [`response_mime_type`](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#:~:text=in%20the%20list.-,responseMimeType,-(Preview)) to get the response in JSON format.

In [9]:
sheet_music_pdf_uri = "gs://github-repo/use-cases/sheet-music/SebbenCrudele.pdf"
output_file_name = "SebbenCrudele.pdf"

sheet_music_extraction_prompt = """The following document is a piece of sheet music. Your task is to output structured metadata about the piece of music in the document. Correct any mistakes that are in the document and fill in missing information when not present in the document.

Output the data in the following JSON format:

{
    "/Title": "Title of the piece",
    "/Author": "Composer(s) of the piece",
    "/Subject": "Music Genre(s) in a comma separated list",
}

"""

# Load file directly from Google Cloud Storage
file_part = Part.from_uri(
    file_uri=sheet_music_pdf_uri,
    mime_type="application/pdf",
)

generation_config = GenerateContentConfig(
    temperature=1, response_mime_type="application/json"
)

# Send to Gemini
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[sheet_music_extraction_prompt, file_part],
    config=generation_config,
)

# Display results
display(Markdown(response.text))

new_metadata = json.loads(response.text)

{
    "/Title": "Sebben, crudele",
    "/Author": "Antonio Caldara",
    "/Subject": "Canzonetta"
}

Next, we'll download the PDF from the GCS Bucket and edit the metadata using the [`PyPDF2`](https://pypdf2.readthedocs.io/en/3.x/) library.

In [10]:
! gcloud storage cp {sheet_music_pdf_uri} {output_file_name}

Copying gs://github-repo/use-cases/sheet-music/SebbenCrudele.pdf to file://SebbenCrudele.pdf
  Completed files 1/1 | 586.7kiB/586.7kiB                                      

Average throughput: 38.9MiB/s


In [11]:
def edit_pdf_metadata(file_path: str, new_metadata: dict) -> None:
    """Edits metadata of a PDF file.

    Args:
        file_path (str): Path to the PDF file.
        new_metadata (dict): Dictionary containing the new metadata fields and values.
            Example: {'/Author': 'John Doe', '/Title': 'My Report'}
    """
    with open(file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_writer = PyPDF2.PdfWriter()

        pdf_writer.clone_reader_document_root(pdf_reader)
        pdf_writer.add_metadata(new_metadata)

        with open(file_path, "wb") as out_file:
            pdf_writer.write(out_file)


edit_pdf_metadata(output_file_name, new_metadata)

In [12]:
pdf_reader = PyPDF2.PdfReader(output_file_name)
print(pdf_reader.metadata)

{'/Producer': 'PyPDF2', '/Title': 'Sebben, crudele', '/Author': 'Antonio Caldara', '/Subject': 'Canzonetta'}
